In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import dataframe_image as dfi
from datetime import datetime
import sklearn.metrics as mtr
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Função que cria os Modelos

In [58]:
def cria_modelo(df_teste, x_teste, y_teste, x_treino, y_treino):

    #Treino do Modelo XGB e salvamento do tempo
    inicio_tempo_treino_xgb = datetime.now()
    modelo_xgb = xgb.XGBRegressor(gamma = 0.0,
    learning_rate = 0.01,
    max_depth = 4,
    min_child_weight = 8,
    n_estimators = 500)
    modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)])

    fim_tempo_treino_xgb = datetime.now()
    tempo_treino_xgb = fim_tempo_treino_xgb - inicio_tempo_treino_xgb
    tempo_treino_xgb = float(f"{str(tempo_treino_xgb.seconds)}.{str(tempo_treino_xgb.microseconds)}")

    #Treino do Modelo RF e salvamento do tempo
    inicio_tempo_treino_rf = datetime.now()
    modelo_rf = RandomForestRegressor(bootstrap = True,
    max_depth = 11,
    max_features = 'log2',
    min_samples_leaf = 7,
    min_samples_split = 6,
    n_estimators = 500)
    modelo_rf.fit(x_treino, y_treino)

    fim_tempo_treino_rf = datetime.now()
    tempo_treino_rf = fim_tempo_treino_rf - inicio_tempo_treino_rf
    tempo_treino_rf = float(f"{str(tempo_treino_rf.seconds)}.{str(tempo_treino_rf.microseconds)}")

    #Treino do Modelo LR e salvamento do tempo
    inicio_tempo_treino_lr = datetime.now()
    modelo_lr = LinearRegression()
    modelo_lr.fit(x_treino, y_treino)

    fim_tempo_treino_lr = datetime.now()
    tempo_treino_lr = fim_tempo_treino_lr - inicio_tempo_treino_lr
    tempo_treino_lr = float(f"{str(tempo_treino_lr.seconds)}.{str(tempo_treino_lr.microseconds)}")

    #Nomeando a coluna que receberá as previsões
    TesteNomeXGB = f"XGB - {TesteNome}"
    TesteNomeRF = f"RF - {TesteNome}"
    TesteNomeLR = f"LR - {TesteNome}"

    #Predição do XGB e salvamento do tempo
    inicio_tempo_previsao_xgb = datetime.now()
    df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
    fim_tempo_previsao_xgb = datetime.now()
    tempo_previsao_xgb = fim_tempo_previsao_xgb - inicio_tempo_previsao_xgb
    tempo_previsao_xgb = float(f"{str(tempo_previsao_xgb.seconds)}.{str(tempo_previsao_xgb.microseconds)}")

    #Predição do RF e salvamento do tempo
    inicio_tempo_previsao_rf = datetime.now()
    df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
    fim_tempo_previsao_rf = datetime.now()
    tempo_previsao_rf = fim_tempo_previsao_rf - inicio_tempo_previsao_rf
    tempo_previsao_rf = float(f"{str(tempo_previsao_rf.seconds)}.{str(tempo_previsao_rf.microseconds)}")

    #Predição do LR e salvamento do tempo
    inicio_tempo_previsao_lr = datetime.now()
    df_teste[f'{TesteNomeLR}'] = modelo_lr.predict(x_teste)
    fim_tempo_previsao_lr = datetime.now()
    tempo_previsao_lr = fim_tempo_previsao_lr - inicio_tempo_previsao_lr
    tempo_previsao_lr = float(f"{str(tempo_previsao_lr.seconds)}.{str(tempo_previsao_lr.microseconds)}")

    #Métricas do XGB
    RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeXGB}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeXGB}"])

    #Métricas do RF
    RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeRF}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeRF}"])

    #Métricas do LR
    RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeLR}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeLR}"])

    #Salvando as métricas num DataFrame Bonitinho
    metrics.append([f'{TesteNome}', Rota,'XGBoost','RRSE',RRSE_xgb, tempo_treino_xgb, tempo_previsao_xgb])
    metrics.append([f'{TesteNome}', Rota,'XGBoost','MAPE',MAPE_xgb, tempo_treino_xgb, tempo_previsao_xgb])
    metrics.append([f'{TesteNome}', Rota,'Random Forest','RRSE',RRSE_rf, tempo_treino_rf, tempo_previsao_rf])
    metrics.append([f'{TesteNome}', Rota,'Random Forest','MAPE',MAPE_rf, tempo_treino_rf, tempo_previsao_rf])
    metrics.append([f'{TesteNome}', Rota,'Linear Regression','RRSE',RRSE_lr, tempo_treino_lr, tempo_previsao_lr])
    metrics.append([f'{TesteNome}', Rota,'Linear Regression','MAPE',MAPE_lr, tempo_treino_lr, tempo_previsao_lr])

    #Salvando as predições num DataFrame Bonitinho
    for i in range(df_teste.shape[0]):
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'XGBoost',df_teste[f"{TesteNomeXGB}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Random Forest',df_teste[f"{TesteNomeRF}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Linear Regression',df_teste[f"{TesteNomeLR}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Tempo real',df_teste["tempo_viagem"].iloc[i]])

    # fi_xgb = pd.DataFrame(data=modelo_xgb.feature_importances_, index=modelo_xgb.feature_names_in_, columns=["importance"])
    # fi_xgb.sort_values("importance").plot(kind="barh", title="Importância Dados XGB")
    # plt.savefig(f"Imagens/Feature Importance/FI_XGB_{TesteNome}.jpg")

    # fi_rf = pd.DataFrame(data=modelo_rf.feature_importances_, index=x_treino.columns, columns=["importance"])
    # fi_rf.sort_values("importance").plot(kind="barh", title="Importância Dados RF")
    # plt.savefig(f"Imagens/Feature Importance/FI_RF_{TesteNome}.jpg")

    # coefs_lr = pd.DataFrame(modelo_lr.coef_, columns=['Coeficiente'], index = x_treino.columns)
    # coefs_lr.Coeficiente = coefs_lr.Coeficiente.abs()
    # coefs_lr['DesvioPadrão'] = x_treino.std(axis=0)
    # coefs_lr['Importancia'] = (100*coefs_lr['DesvioPadrão']*coefs_lr['Coeficiente'])/coefs_lr['DesvioPadrão']*coefs_lr['Coeficiente'].max()
    # coefs_lr['Importancia'].plot(kind='barh', title = 'Importância Dados LR')
    # plt.savefig(f"Imagens/Feature Importance/FI_LR_{TesteNome}.jpg")
    

# Parte pra editar os teste

In [59]:
df = pd.read_csv('Clean/rota33642.csv')
df.head()
Rota = "SP - 33642"
predictions = []
metrics = []

In [60]:
# metrics = pd.DataFrame(metrics, columns=['Teste','Rota','Modelo','Métrica','Valor', 'Tempo de treino', 'Tempo de previsão'])
# predictions = pd.DataFrame(predictions, columns=['Teste','Data','Rota','Modelo','Valor'])
# metrics.to_csv('metrics.csv', index = False)
# predictions.to_csv('predictions.csv', index = False)

In [61]:
Colunas = [
    #Primeira parte teste de colunas
    ["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "hora_dia"],
    ["dia_semana", "tipo_dia", "turno_dia"],
    ["dia_semana", "tipo_dia", "hora_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "turno_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "hora_dia","partidaTimeStamp"],
    ["dia_semana", "tipo_dia", "turno_dia","partidaTimeStamp"],

    #Segunda parte teste com dados de clima
    ["dia_semana", "tipo_dia", "hora_dia", "Precipitacao", "Temperatura"],
    ["dia_semana", "tipo_dia", "hora_dia", "Chuva", "Calor"],
    ["dia_semana", "tipo_dia", "hora_dia", "Precipitacao"],
    ["dia_semana", "tipo_dia", "hora_dia", "Temperatura"],
    ["dia_semana", "tipo_dia", "hora_dia", "Chuva"],
    ["dia_semana", "tipo_dia", "hora_dia", "Calor"],

    #Terceira parte teste com os tempo de viagens anteriores
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4", "tempo_viagem_5"],

    #Quarta parte alternar percentual de treino
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]
    
]

TestesNome = [
    #Primeira parte teste de colunas
    "0 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, turno_dia, qtdDiasAno, partidaTimeStamp]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, qtdDiasAno]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia, qtdDiasAno]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, partidaTimeStamp]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia, partidaTimeStamp]",

    #Segunda parte teste com dados de clima
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Precipitacao, Temperatura]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Chuva, Calor]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Precipitacao]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Temperatura]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Chuva]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Calor]",

    #Terceira parte teste com os tempo de viagens anteriores
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3, tempo_viagem_4]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3, tempo_viagem_4, tempo_viagem_5]",

    #Quarta parte alternar percentual de treino
    "4 Teste de quantidade de dados para treino - 2 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 4 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 6 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 8 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 10 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]"

]

PercentTeste = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.152173, 0.304347, 0.456521, 0.608695, 0.760869]


In [62]:
print(f'{len(Colunas)} --- {len(TestesNome)} --- {len(PercentTeste)}')

23 --- 23 --- 23


In [63]:
#for i in range(9,10,1):
for i in range(len(TestesNome)):
    shape = int(df.shape[0] * (1-PercentTeste[i]))
    df = df.sort_values(by=["data_partida"], ascending=False)
    df_teste = df[:shape]
    df_treino = df[shape:]

    TesteNome = TestesNome[i]

    x_teste = df_teste[Colunas[i]]
    y_teste = df_teste["tempo_viagem"]
    x_treino = df_treino[Colunas[i]]
    y_treino = df_treino["tempo_viagem"]

    cria_modelo(df_teste, x_teste, y_teste, x_treino, y_treino)

[0]	validation_0-rmse:1171.59476
[1]	validation_0-rmse:1160.12532
[2]	validation_0-rmse:1148.77271
[3]	validation_0-rmse:1137.53575
[4]	validation_0-rmse:1126.41459
[5]	validation_0-rmse:1115.40529
[6]	validation_0-rmse:1104.50956
[7]	validation_0-rmse:1093.72349
[8]	validation_0-rmse:1083.04881
[9]	validation_0-rmse:1072.48316
[10]	validation_0-rmse:1062.02367
[11]	validation_0-rmse:1051.67262
[12]	validation_0-rmse:1041.42561
[13]	validation_0-rmse:1031.28503
[14]	validation_0-rmse:1021.24641
[15]	validation_0-rmse:1011.31217
[16]	validation_0-rmse:1001.47790
[17]	validation_0-rmse:991.74599
[18]	validation_0-rmse:982.11207
[19]	validation_0-rmse:972.57857
[20]	validation_0-rmse:963.14109
[21]	validation_0-rmse:953.80213
[22]	validation_0-rmse:944.55728
[23]	validation_0-rmse:935.40907
[24]	validation_0-rmse:926.35313
[25]	validation_0-rmse:917.39195
[26]	validation_0-rmse:908.52300
[27]	validation_0-rmse:899.74351
[28]	validation_0-rmse:891.05620
[29]	validation_0-rmse:882.45658
[30

C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1171.59476
[1]	validation_0-rmse:1160.12532
[2]	validation_0-rmse:1148.77271
[3]	validation_0-rmse:1137.53575
[4]	validation_0-rmse:1126.41459
[5]	validation_0-rmse:1115.40529
[6]	validation_0-rmse:1104.50956
[7]	validation_0-rmse:1093.72349
[8]	validation_0-rmse:1083.04881
[9]	validation_0-rmse:1072.48316
[10]	validation_0-rmse:1062.02367
[11]	validation_0-rmse:1051.67262
[12]	validation_0-rmse:1041.42561
[13]	validation_0-rmse:1031.28503
[14]	validation_0-rmse:1021.24641
[15]	validation_0-rmse:1011.31217
[16]	validation_0-rmse:1001.47790
[17]	validation_0-rmse:991.74599
[18]	validation_0-rmse:982.11207
[19]	validation_0-rmse:972.57857
[20]	validation_0-rmse:963.14109
[21]	validation_0-rmse:953.80213
[22]	validation_0-rmse:944.55728
[23]	validation_0-rmse:935.40907
[24]	validation_0-rmse:926.35313
[25]	validation_0-rmse:917.39195
[26]	validation_0-rmse:908.52300
[27]	validation_0-rmse:899.74351
[28]	validation_0-rmse:891.05623
[29]	validation_0-rmse:882.45664
[30

C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1171.69540
[1]	validation_0-rmse:1160.32665
[2]	validation_0-rmse:1149.07483
[3]	validation_0-rmse:1137.93874
[4]	validation_0-rmse:1126.91726
[5]	validation_0-rmse:1116.00929
[6]	validation_0-rmse:1105.21378
[7]	validation_0-rmse:1094.52951
[8]	validation_0-rmse:1083.95552
[9]	validation_0-rmse:1073.49068
[10]	validation_0-rmse:1063.13393
[11]	validation_0-rmse:1052.88428
[12]	validation_0-rmse:1042.74061
[13]	validation_0-rmse:1032.70194
[14]	validation_0-rmse:1022.76724
[15]	validation_0-rmse:1012.93554
[16]	validation_0-rmse:1003.20580
[17]	validation_0-rmse:993.57707
[18]	validation_0-rmse:984.04833
[19]	validation_0-rmse:974.61867
[20]	validation_0-rmse:965.28711
[21]	validation_0-rmse:956.05266
[22]	validation_0-rmse:946.91444
[23]	validation_0-rmse:937.87150
[24]	validation_0-rmse:928.92292
[25]	validation_0-rmse:920.06783
[26]	validation_0-rmse:911.30528
[27]	validation_0-rmse:902.63442
[28]	validation_0-rmse:894.05433
[29]	validation_0-rmse:885.56418
[30

...

C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1173.01600
[1]	validation_0-rmse:1161.55847
[2]	validation_0-rmse:1150.21727
[3]	validation_0-rmse:1138.99203
[4]	validation_0-rmse:1127.88092
[5]	validation_0-rmse:1116.88356
[6]	validation_0-rmse:1105.99742
[7]	validation_0-rmse:1095.22336
[8]	validation_0-rmse:1084.55833
[9]	validation_0-rmse:1074.00279
[10]	validation_0-rmse:1063.55541
[11]	validation_0-rmse:1053.21399
[12]	validation_0-rmse:1042.97907
[13]	validation_0-rmse:1032.84796
[14]	validation_0-rmse:1022.82132
[15]	validation_0-rmse:1012.89646
[16]	validation_0-rmse:1003.07496
[17]	validation_0-rmse:993.35185
[18]	validation_0-rmse:983.73162
[19]	validation_0-rmse:974.20664
[20]	validation_0-rmse:964.78200
[21]	validation_0-rmse:955.45007
[22]	validation_0-rmse:946.21782
[23]	validation_0-rmse:937.07416
[24]	validation_0-rmse:928.02422
[25]	validation_0-rmse:919.07322
[26]	validation_0-rmse:910.20831
[27]	validation_0-rmse:901.44048
[28]	validation_0-rmse:892.75724
[29]	validation_0-rmse:884.16900
[30

C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1172.79726
[1]	validation_0-rmse:1161.33269
[2]	validation_0-rmse:1149.98495
[3]	validation_0-rmse:1138.75293
[4]	validation_0-rmse:1127.63554
[5]	validation_0-rmse:1116.63151
[6]	validation_0-rmse:1105.73986
[7]	validation_0-rmse:1094.95934
[8]	validation_0-rmse:1084.28807
[9]	validation_0-rmse:1073.72528
[10]	validation_0-rmse:1063.27144
[11]	validation_0-rmse:1052.91964
[12]	validation_0-rmse:1042.67741
[13]	validation_0-rmse:1032.53601
[14]	validation_0-rmse:1022.49846
[15]	validation_0-rmse:1012.56603
[16]	validation_0-rmse:1002.73267
[17]	validation_0-rmse:993.00237
[18]	validation_0-rmse:983.36921
[19]	validation_0-rmse:973.83494
[20]	validation_0-rmse:964.39957
[21]	validation_0-rmse:955.06318
[22]	validation_0-rmse:945.81845
[23]	validation_0-rmse:936.66930
[24]	validation_0-rmse:927.61564
[25]	validation_0-rmse:918.65146
[26]	validation_0-rmse:909.78446
[27]	validation_0-rmse:901.00576
[28]	validation_0-rmse:892.31787
[29]	validation_0-rmse:883.71772
[30

C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_18804\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

# Melhor Métrica

In [18]:
df = pd.read_csv('metrics.csv')
df.head(20)

,Teste,Rota,Modelo,Métrica,Valor,Tempo de treino,Tempo de previsão
0,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,XGBoost,RRSE,0.704792,1.719387,0.700000
1,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,XGBoost,MAPE,0.108865,1.719387,0.700000
2,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,Random Forest,RRSE,0.713155,1.553346,0.112998
3,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,Random Forest,MAPE,0.107034,1.553346,0.112998
4,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,Linear Regression,RRSE,0.939192,0.300100,0.200000
5,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,Linear Regression,MAPE,0.153611,0.300100,0.200000
6,"1 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,XGBoost,RRSE,0.687590,0.968598,0.497000
7,"1 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,XGBoost,MAPE,0.100577,0.968598,0.497000
8,"1 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,Random Forest,RRSE,0.684188,0.839213,0.113019
9,"1 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,Random Forest,MAPE,0.101539,0.839213,0.113019


In [30]:
min_a = df.loc[df['Métrica'] == 'RRSE','Valor'].min()
a = df.loc[df['Valor'] == min_a]
a

,Teste,Rota,Modelo,Métrica,Valor,Tempo de treino,Tempo de previsão
96,3 Teste com os tempos de viagem anteriores - [...,SP - 33642,XGBoost,RRSE,0.682129,1.202785,0.5


In [31]:
min_b = df.loc[df['Métrica'] == 'MAPE','Valor'].min()
b = df.loc[df['Valor'] == min_b]
b

,Teste,Rota,Modelo,Métrica,Valor,Tempo de treino,Tempo de previsão
7,"1 Teste de Colunas - [dia_semana, tipo_dia, ho...",SP - 33642,XGBoost,MAPE,0.100577,0.968598,0.497
